In [1]:
import scrapy, re, urllib, datetime, io, requests
import pandas as pd
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
from bs4 import BeautifulSoup as bs

In [ ]:
class item_coaches(scrapy.Item):
    """Scrapy Item for Coaching Profiles"""
    Year = scrapy.Field()
    Org = scrapy.Field()
    Division = scrapy.Field()
    Wins = scrapy.Field()
    Losses = scrapy.Field()
    Ties = scrapy.Field()
    WL = scrapy.Field()
    Notes = scrapy.Field()
    Name = scapy.Field()

In [ ]:
class CoachesCSVPipeline(object):
    """Scrapy Pipeline Object for Coaching Profiles"""
    def __init__(self):
        fileName = "Coaches.csv"
        self.file = open(fileName, 'wb')
        self.exporter = CsvItemExporter(self.file, fields_to_export=["Name", "Year", "Org", "Division", "Wins", "Losses", "Ties", "WL", "Notes"])
        self.exporter.start_exporting()
 
    def close_spider(self, spider):
        self.exporter.finish_exporting()
        self.file.close()
 
    def process_item(self, item, spider):
        self.exporter.export_item(item)
        return item

In [2]:
class CoachSpider(scrapy.Spider): 
    name = "Coach"
    
    # Read in data
    data = pd.read_csv(file_in)
    df = data[data.key == target].drop_duplicates()
    df.sort_values(['team'], inplace = True)
    _list = df.to_dict(orient='record')
    
    def start_requests(self): 
        
        for url in self._list[:1]: 
            yield scrapy.Request(url=url['link'], 
                                 callback=self.parse, 
                                 meta={'name':url['txt']}
                                )
    
    def parse(self, response):         
        # Create table 
        tables = pd.read_html(response.body)
        tables[1]['Name'] = response.meta['name']

        # Convert the table to a dict and yield those results to items 
        
        

In [3]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(CoachSpider)
process.start() 

2017-12-05 21:54:02 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-05 21:54:02 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-12-05 21:54:02 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-05 21:54:02 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

In [2]:
class CoachSpider(scrapy.Spider): 
    name = "Coach"
    
    file_in = "NewLinks2017-12-03 20:28:18.862583.csv"
    file_out = "Coaches_{0}.csv"
    variables = [ "Year", "Org", "Division", "Wins", "Losses", "Ties", "WL%", "Notes", "Name"]
    header = ','.join(variables) + "\n"
    target = 'people'
    
    # Read in data
    data = pd.read_csv(file_in)
    df = data[data.key == target].drop_duplicates()
    df.sort_values(['team'], inplace = True)
    _list = df.to_dict(orient='record')
    
    # Create file to append to 
    fileName = file_out.format(datetime.datetime.now())
    
    with open(fileName, 'w') as f: 
        f.write(header)
    
    def start_requests(self): 
        
        for url in self._list[:1]: 
            yield scrapy.Request(url=url['link'], 
                                 callback=self.parse, 
                                 meta={'name':url['txt']}
                                )
    
    def parse(self, response):         
        # Create tables 
        tables = pd.read_html(response.body)
        tables[1]['Name'] = response.meta['name']

        # Create a pandas df and append to csv 
        c = io.StringIO()
        tables[1][self.variables][:-1].to_csv(c, index=False, header=False)
        
        # Append links to CSV 
        with open(self.fileName, 'a') as f:
            f.write(c.getvalue())
        